
# Dual-Stream + Anti-Collapse: Colab/Kaggle Ready Walkthrough

This notebook reconstructs the repository assets at runtime, installs the dependencies, and runs both halves of the project:

- **Dual-Stream generation** with a compact Hugging Face model to produce the Answer and Monologue streams.
- **Anti-collapse monitoring**: train a baseline classifier, evaluate a new batch for drift/performance/outliers, and audit Dual-Stream traces for coherence.

All code is wired to run end-to-end on CPU inside Colab or Kaggle without any placeholder steps.


In [ ]:

%pip install -q --upgrade pip
%pip install -q --upgrade pandas numpy scikit-learn scipy joblib tqdm transformers torch matplotlib


In [ ]:

import base64, io, tarfile, sys
from pathlib import Path

ARCHIVE_B64 = "..."

def ensure_repo_root() -> Path:
    candidates = [
        Path.cwd(),
        Path("/content/dual-stream"),
        Path("/kaggle/working/dual-stream"),
        Path.home() / "dual-stream",
    ]
    for c in candidates:
        if (c / "dualstream_anticollapse").exists() and (c / "python_poc").exists():
            return c.resolve()

    target = Path("/kaggle/working/dual-stream").resolve()
    target.mkdir(parents=True, exist_ok=True)
    data = base64.b64decode(ARCHIVE_B64)
    with tarfile.open(fileobj=io.BytesIO(data), mode="r:gz") as tar:
        tar.extractall(path=target)
    return target

REPO_DIR = ensure_repo_root()
print(f"Using repo root: {REPO_DIR}")
for extra in [REPO_DIR, REPO_DIR / "dualstream_anticollapse", REPO_DIR / "python_poc"]:
    if str(extra) not in sys.path:
        sys.path.append(str(extra))

artifact_dir = REPO_DIR / "artifacts_demo"
artifact_dir.mkdir(parents=True, exist_ok=True)


In [ ]:

from pathlib import Path
print("dualstream_anticollapse files:", sorted(p.name for p in (REPO_DIR / "dualstream_anticollapse").iterdir()))
print("python_poc files:", sorted(p.name for p in (REPO_DIR / "python_poc").iterdir()))


## 1. Generate Dual-Stream output (Answer + Monologue)

In [ ]:

    from python_poc.dual_stream_poc import DualStream
    import json

    prompt = "Explain how a dual-stream safety monitor helps catch misalignment before deployment."

    # Use a compact model so it runs quickly on CPU in hosted notebooks
    streamer = DualStream(model_name="sshleifer/tiny-gpt2", top_k=5)
    result = streamer.generate(prompt=prompt, max_new_tokens=60, temperature=0.8, top_p=0.95)

    print("Answer Stream (trimmed):
", result["answer_text"])
    print("
First few monologue frames:")
    for line in result["monologue_text"].splitlines()[:5]:
        print(line)

    out_path = artifact_dir / "dual_stream_output.json"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w") as f:
        json.dump(result, f, indent=2)
    print(f"
Saved full dual-stream payload to: {out_path}")


## 2. Train baseline and monitor a current batch

In [ ]:

from dualstream_anticollapse.cli import cmd_train
from types import SimpleNamespace

train_args = SimpleNamespace(
    train_csv=str(REPO_DIR / "dualstream_anticollapse/demo/reference.csv"),
    target="y",
    id_column=None,
    features="x1,x2",
    model_type="sgd_classifier",
    artifacts=str(artifact_dir),
)
cmd_train(train_args)


In [ ]:

from dualstream_anticollapse.cli import cmd_monitor
from types import SimpleNamespace

monitor_args = SimpleNamespace(
    reference_csv=str(REPO_DIR / "dualstream_anticollapse/demo/reference.csv"),
    current_csv=str(REPO_DIR / "dualstream_anticollapse/demo/current.csv"),
    target="y",
    id_column=None,
    features="x1,x2",
    model_type="sgd_classifier",
    artifacts=str(artifact_dir),
)
cmd_monitor(monitor_args)


## 3. Audit Dual-Stream traces for coherence risk

In [ ]:

from dualstream_anticollapse.cli import cmd_audit_dual
from types import SimpleNamespace
import json

audit_args = SimpleNamespace(
    dual_jsonl=str(REPO_DIR / "dualstream_anticollapse/demo/dual_stream_sample.jsonl"),
    target="y",
    id_column=None,
    artifacts=str(artifact_dir),
)
cmd_audit_dual(audit_args)

report_path = artifact_dir / "coherence_report.json"
report = json.load(open(report_path))
print(f"Loaded {len(report)} audit rows; sample:")
print(json.dumps(report[:2], indent=2))


## 4. Run the coherence auditor on the freshly generated dual-stream output

In [ ]:

from dualstream_anticollapse.coherence import CoherenceAuditor
from dualstream_anticollapse.config import Config
import json

config = Config(target="y")
auditor = CoherenceAuditor(thresholds=config.thresholds.__dict__)

logits_topk = []
first_frame = result["monologue_frames"][0]
for tid, prob in zip(first_frame["topk_ids"], first_frame["topk_probs"]):
    tok = streamer.tokenizer.decode([tid]).strip() or str(tid)
    logits_topk.append((tok, prob))

generated_record = {
    "answer": result["answer_text"],
    "monologue": result["monologue_text"],
    "logits_topk": logits_topk,
}
coherence_out = auditor.audit_record(generated_record)
print(json.dumps(coherence_out, indent=2))



### What happened?

- **Dual-Stream generation** produced synchronized Answer/Monologue channels using a lightweight GPT-2 model.
- **Training + monitoring** fit a baseline classifier on the reference CSV, then checked the current batch for drift, outliers, and performance drops.
- **Coherence auditing** flagged any deception, conflict, or safety markers inside the monologue trace, including the one we just generated.

Artifacts land in `artifacts_demo/` so you can download them from Colab/Kaggle or feed them into your own pipelines.
